# Introduction to NLP Fundamentals in Tensorflow

NLP has the goal of deriving information out of natural language (could be sequences text or speech)

Another common term for NLP problems is sequence to sequence problems (seq2seq).

## Check for GPU

In [1]:
!nvidia-smi -L

NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.



## Get helper functions


In [2]:
!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py

# Import series of helper functions for the notebook

from helper_functions import unzip_data, create_tensorboard_callback, plot_loss_curves, compare_historys

--2021-10-10 00:42:09--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0s      

2021-10-10 00:42:09 (92.3 MB/s) - ‘helper_functions.py’ saved [10246/10246]



## Get a text dataset

The dataset we're going to be using is Kaggle Introduction to NLP dataset (text samples of Tweets labelled as dister or not disaster).

In [3]:
!wget https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip

# Unzip data
unzip_data("nlp_getting_started.zip")

--2021-10-10 00:42:12--  https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 108.177.97.128, 108.177.125.128, 142.250.157.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|108.177.97.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 607343 (593K) [application/zip]
Saving to: ‘nlp_getting_started.zip’

nlp_getting_started 100%[===================>] 593.11K  --.-KB/s    in 0.005s  

2021-10-10 00:42:12 (118 MB/s) - ‘nlp_getting_started.zip’ saved [607343/607343]



## Visualizing a text dataset

To visualize our text samples, we first have to read them in, one way to do would be to use Python. 

But I prefer to get visual straight away.

So another way to do this is to use pandas...

In [4]:
import pandas as pd
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')
train_df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [5]:
# Shuffle trianing dataframe
train_df_shuffled = train_df.sample(frac=1, random_state=42)
train_df_shuffled.head()

,id,keyword,location,text,target
2644,3796,destruction,NaN,So you have a new weapon that can cause un-ima...,1
2227,3185,deluge,NaN,The f$&amp;@ing things I do for #GISHWHES Just...,0
5448,7769,police,UK,DT @georgegalloway: RT @Galloway4Mayor: ÛÏThe...,1
132,191,aftershock,NaN,Aftershock back to school kick off was great. ...,0
6845,9810,trauma,"Montgomery County, MD",in response to trauma Children of Addicts deve...,0


In [6]:
# What does the test dataframe look like
test_df.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [7]:
# How many examples of each class?
train_df.target.value_counts(normalize=True)

0    0.57034
1    0.42966
Name: target, dtype: float64

In [8]:
# Let's visualize some random training examples
import random
random_index = random.randint(0, len(train_df)-5) # create random indexes not higher that total num samples
for row in train_df_shuffled[["text", "target"]][random_index:random_index+5].itertuples():
    _, text, target = row
    print(f'Target: {target}', '(real disaster)' if target > 0 else '(not real disaster)')
    print(f'Text:\n{text}\n')
    print('---\n')

Target: 1 (real disaster)
Text:
@gigagil IOF murdered over 513 Palestinian children (70% under 12) during Gaza Massacre where was zionist morality-zionism is a world evil!

---

Target: 1 (real disaster)
Text:
Christian Attacked by Muslims at the Temple Mount after Waving Israeli Flag via Pamela Geller - ... http://t.co/e4wK8Uri8A

---

Target: 1 (real disaster)
Text:
Rly tragedy in MP: Some live to recount horror: ÛÏWhen I saw coaches of my train plunging into water I called ... http://t.co/xtlJz7BjgL

---

Target: 1 (real disaster)
Text:
Tennessee lesbian couple faked hate crime and destroyed own home with arsonÛ_ http://t.co/10mUEY8PXJ #Lesbian

---

Target: 0 (not real disaster)
Text:
Man Who Buried Dog Alive Thought No One Would Find Her But She Was Rescued Just In Time http://t.co/SahQ5UOAHW

---



## Split data into trianing and validation sets

In [9]:
from sklearn.model_selection import train_test_split

In [10]:
# Use train_test_split to split trianing data into training and validation sets
train_sentences, val_sentences, train_labels, val_labels = train_test_split(train_df_shuffled['text'],
                                                                            train_df_shuffled['target'],
                                                                            test_size = 0.1, # use 10% data for val split
                                                                            random_state = 42) 

In [11]:
# Check the lenghts
len(train_sentences), len(train_labels), len(val_sentences), len(val_labels)

(6851, 6851, 762, 762)

In [12]:
# Check the first 10 samples
train_sentences[:10], train_labels[:10]

(5921    @mogacola @zamtriossu i screamed after hitting...
 3904              Imagine getting flattened by Kurt Zouma
 2804    @Gurmeetramrahim #MSGDoing111WelfareWorks Gree...
 3718    @shakjn @C7 @Magnums im shaking in fear he's g...
 1667    Somehow find you and I collide http://t.co/Ee8...
 4435    @EvaHanderek @MarleyKnysh great times until th...
 2544                     destroy the free fandom honestly
 7223    Weapons stolen from National Guard Armory in N...
 4265    @wfaaweather Pete when will the heat wave pass...
 6568    Patient-reported outcomes in long-term survivo...
 Name: text, dtype: object, 5921    0
 3904    0
 2804    1
 3718    0
 1667    0
 4435    1
 2544    1
 7223    0
 4265    1
 6568    1
 Name: target, dtype: int64)

## Converting text to numbers

When dealing with text problem, one of the first things you'll have to do before you can build a model is to convert your text to numbers.

There are few ways to do this, namely:
* Tokenization - direct mapping of token (a token could be a word or a character to number)
* Embedding - Create a matrix of feature vector to each token (the size of the feature vector can be defined adn this embedding can be learned). 

### Text vectorization (tokenization)

In [13]:
train_sentences[:5]

5921    @mogacola @zamtriossu i screamed after hitting...
3904              Imagine getting flattened by Kurt Zouma
2804    @Gurmeetramrahim #MSGDoing111WelfareWorks Gree...
3718    @shakjn @C7 @Magnums im shaking in fear he's g...
1667    Somehow find you and I collide http://t.co/Ee8...
Name: text, dtype: object

In [14]:
import tensorflow as tf
from tensorflow.keras.layers import TextVectorization

# Use the default TextVectorization parameters
text_vectorizer = TextVectorization(max_tokens=None, # how many words in the vocabulary (automatically add <OOV>)
                                    standardize='lower_and_strip_punctuation',
                                    split='whitespace',
                                    ngrams=None, # Create group of n-words
                                    output_mode='int', # how to map token to numbers
                                    output_sequence_length=None, # how long do you want your sequences to be?
                                    )

In [15]:
# Find the average number of tokens (words) int the trianing tweets
round(sum([len(i.split()) for i in train_sentences])/len(train_sentences))

15

In [16]:
# Setup text vectorization variables
max_vocab_length = 10000 # max number of words to ahve in our vocabulary
max_length = 15 # max length our sequences will be (e.g. how many words from a Tweet does a model see?)

text_vectorizer = TextVectorization(max_tokens=max_vocab_length,
                                    output_mode='int',
                                    output_sequence_length=max_length)

In [17]:
# Fit the text vectorizer to the training text
text_vectorizer.adapt(train_sentences)

In [18]:
# Create a sample sentence and tokenize it
sample_sentence = "There's a flood in my street!"
text_vectorizer([sample_sentence])

<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[264,   3, 232,   4,  13, 698,   0,   0,   0,   0,   0,   0,   0,
          0,   0]])>

In [19]:
# Choose a ranodm sentence from training dataset and tokenize it
random_sentence = random.choice(train_sentences)
print(f'Original text: \n {random_sentence} \
      \n \nVectorized version:')
text_vectorizer([random_sentence])

Original text: 
 @kelworldpeace @TAXSTONE yoga flames!       
 
Vectorized version:


<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[   1, 7776, 6552,  218,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0]])>

In [20]:
# Get the unique words in the vocabulary
words_in_vocab = text_vectorizer.get_vocabulary() # get all the unique words in our training data
top_5_words = words_in_vocab[:5] # get the most common words
bottom_5_words  = words_in_vocab[-5:] # least common words
print(f'Number of words in vocab:{len(words_in_vocab)}')
print(f'5 most common words: {top_5_words}')
print(f'5 least common words: {bottom_5_words}')

Number of words in vocab:10000
5 most common words: ['', '[UNK]', 'the', 'a', 'in']
5 least common words: ['pages', 'paeds', 'pads', 'padres', 'paddytomlinson1']


### Creating and Embedding using an Embedding Layer

To make our emebedding, we're going to user TF Embedding Layer.

The parameters we care most abour for our embedding layer:
* `input_dim` = The size of our vocabulary
* `output_dim` = The size of the output embedding vector, for example a value of a 100 would mean each token gets represented by a vector of 100 long.
* `input_length` = length of sequences being passed to the embedding layer.



In [24]:
from tensorflow.keras import layers

embedding = layers.Embedding(input_dim = max_vocab_length, # set input shape
                             output_dim = 128, # output shape
                             embeddings_initializer = 'uniform', 
                             input_length = max_length, # how long is each input
                             )

embedding 

In [28]:
# Get a random senteces from the training set
random_sentence = random.choice(train_sentences)
print(f'Original text: \n {random_sentence} \
        \n \n Embedded verison:')

# Embed the random sentence (turn it into dense vector of fixed size)
sample_embed = embedding(text_vectorizer([random_sentence]))
sample_embed

Original text: 
 they say bad things happen for a reason
but no wise words gonna stop te bleeding         
 
 Embedded verison:


<tf.Tensor: shape=(1, 15, 128), dtype=float32, numpy=
array([[[ 0.00758516,  0.04205218,  0.01663078, ..., -0.04234008,
          0.01223342,  0.00709628],
        [-0.00336124, -0.0373435 , -0.04485815, ...,  0.00418804,
          0.04629831, -0.03187019],
        [-0.0311136 , -0.00040649,  0.04930638, ..., -0.03830805,
          0.03791381,  0.01398787],
        ...,
        [-0.0491985 ,  0.00680438, -0.01970023, ..., -0.03935652,
          0.0442407 , -0.04942501],
        [ 0.01891856,  0.03806872, -0.00967699, ...,  0.04110635,
          0.03625362,  0.01153877],
        [ 0.04423399, -0.01433047, -0.01181089, ..., -0.00325406,
          0.02924032,  0.00445372]]], dtype=float32)>

In [29]:
# Check out a single token's embedding
sample_embed[0][0], sample_embed[0][0].shape, random_sentence

(<tf.Tensor: shape=(128,), dtype=float32, numpy=
 array([ 0.00758516,  0.04205218,  0.01663078, -0.00996715, -0.01654432,
         0.04444451,  0.02988956,  0.02649018,  0.04465941,  0.02820157,
         0.03393939, -0.01197755, -0.04140177,  0.02722707, -0.00997461,
         0.04743371,  0.04578072, -0.04338552,  0.01322843, -0.04964966,
         0.00320859,  0.0361498 ,  0.03711443, -0.00955533,  0.04361762,
         0.02706021, -0.03837954,  0.00833241,  0.03135201, -0.03980032,
         0.01597842,  0.03797713, -0.00719221,  0.03023345, -0.03207882,
        -0.04381667,  0.00651171, -0.00139576, -0.03661614, -0.02317587,
         0.00737346, -0.02449852,  0.01475484, -0.03293624,  0.00315577,
        -0.01470459,  0.03531983,  0.03867241, -0.02645   ,  0.00986797,
         0.04554285, -0.01097497,  0.02467301, -0.00822823,  0.04066335,
         0.01149542,  0.00084658, -0.00024178, -0.0155884 , -0.01245417,
         0.01116905,  0.04006055, -0.03368056, -0.02016404, -0.04169778,
  

## Modelling a text dataset (running a series of experiments)

Now we've got a way to turn our text sequences into numbers, it's time to start building a series of modelling experiments.

We'll start with a baseline and move on from there.

* Model 0: Naive Bayes (baseline)
* Model 1: Feed-forward neural network
* Model 2: LSTM model (RNN)
* Model 3: GRU model (RNN)
* Model 4: Bidirectional-LSTM model (RNN)
* Model 5: 1D convolutional neural netwrok (CNN)
* Model 6: Tensorflow Hub Pretrained Feature Extractor (using transfer learning for NLP)
* Model 7: Same as model 6 with 10% of training data.

How are we going to approach all of these?
Use the standard steps in modelling with tensorflow:

* Create a model
* Build a model
* Fit a model 
* Evaluate a model


### Model 0: Getting a baseline

As with all machine learning modelling experiments, it's important to create a baseline model so you've got a becnhmark for future experiments to build upon.

To create our baseline, we'll use Sklearn's Multinomial Naive Bayes using the Tf-idf formula to convert our words to numbers

> 🔑 ** Note:** It's common practice to use non-DL algorithms as a baseline because of their speed and later using DL if you can improve upon them.

In [36]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

In [38]:
# Create tokenization and modelling pipeline
model_0 = Pipeline([
                    ("tfidf", TfidfVectorizer()), #convert words to numbers using tfidf
                    ("clf", MultinomialNB()) # model the text
])

# Fit the pipeline to the training data
model_0.fit(train_sentences, train_labels)

Pipeline(memory=None,
         steps=[('tfidf',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=1.0, max_features=None,
                                 min_df=1, ngram_range=(1, 1), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words=None, strip_accents=None,
                                 sublinear_tf=False,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, use_idf=True,
                                 vocabulary=None)),
                ('clf',
                 MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))],
         verbose=False)

In [42]:
# Evaluate our baseline model
baseline_score = model_0.score(val_sentences, val_labels)
print(f'Our baseline model achieves an accuracy of: {baseline_score*100:.2f}%')

Our baseline model achieves an accuracy of: 79.27%


In [46]:
# Make predictions
baseline_preds = model_0.predict(val_sentences)
baseline_preds[:20]

array([1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1])

In [44]:
train_df.target.value_counts(normalize=True)

0    0.57034
1    0.42966
Name: target, dtype: float64

### Creating an evaluation function for our model experiments

We could evaluate all of our model's prediction with different metrics every time, however, this would be cumbersome and could easily be fixed with a function

Let's create one to compare our model's prediction with the truth label with the following metrics:
* Accuracy
* Precision
* Recall
* F1-Score

In [53]:
# Function to evaluate: accuracy, precision, recall, f1-score
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def calculate_results(y_true, y_pred):
    '''
    Calculates model accuracy, precision, recall and F1-score of binary classification model.
    '''
    # Calculate model accuracy
    model_accuracy = accuracy_score(y_true, y_pred)*100
    # Calculate model precision, recall and F1-score using "weighted" average
    model_precision, model_recall, model_f1, _ = precision_recall_fscore_support(y_true, y_pred, average = 'weighted')
    model_results = {'accuracy':model_accuracy,
                     'precision':model_precision*100,
                     'recall':model_recall*100,
                     'model_f1':model_f1*100}
    return model_results

In [54]:
# Get baseline results
baseline_results = calculate_results(y_true = val_labels,
                                     y_pred = baseline_preds)
baseline_results

{'accuracy': 79.26509186351706,
 'model_f1': 78.6218975804955,
 'precision': 81.11390004213173,
 'recall': 79.26509186351706}